# Model Finetune to Client Traffic post Pruning-SubsetSearch

In this notebook we investigate the potential effects of tha pruning and subset search achieved in the previous step. The aim is to verify whether it is possible to empower the lightweight model with the knwoledge of the oracle model, obtained in the initial Federated Learning training phase.

The main problem is that by pruning and removing features from the network traffic to fit specific deployment scenarios, the organization might have obtained a model with worse performance than the previous one. 
Moreover, if it is willing to specialize that model to its specific traffic categories, how does the model behave against the other previously seen attacks?
Is it possible to preserve the knwoledge? Will it face catastrophic forgetting? Can you postpone catastrophic forgetting somehow?

We will analyse different learning algorithms that the organization can adopt during this finetune process and measure the loss in the global knowledge of the model with respect to all the attacks.

In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import os
import matplotlib as mpl
from sklearn.metrics import accuracy_score
from intellect.model.sklearn.model import EnhancedMlpRegressor
from intellect.io import load, dump, create_dir
from intellect.inspect import set_seed
from intellect.scoring import compute_metric_percategory
from intellect.dataset import (Dataset, indexes_for_oracle_learning)

import config

mpl.rcParams['figure.dpi']= 70
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 20)

# when running local with few cpu uncomment this
import threadpoolctl
threadpoolctl.threadpool_limits(limits=int(os.cpu_count()/4));

In [2]:
train, validation, test = config.get_dataset()
cols = ["Global"] + config.CLIENT_CATEGORIES + [v for v in train._y.value_counts().sort_values(ascending=False).index.values if v not in config.CLIENT_CATEGORIES]

In [7]:
oracle_cache = {}
student_cache = {}

def run_test(save_prefix: str, retrain_ds: Dataset, retrain_val_ds: Dataset, features_available=None,
             skip=False, prune_ratio=None, availability=None, **kwargs):
    if skip is True and os.path.isfile(f"{save_prefix}.csv"):
        return
    if features_available is None:
        features_available = []
    set_seed()
    oracle_net= EnhancedMlpRegressor.load(config.TRAINING_OUTPUT_MODEL)
    student_net = EnhancedMlpRegressor.load(config.TRAINING_OUTPUT_MODEL)
    
    if prune_ratio is not None:
        student_net = config.PRUNE_METHOD(student_net, prune_ratio)

    idx, idx_oracle = indexes_for_oracle_learning(retrain_ds, features_available, availability)
    
    key_oracle = (hash(retrain_ds), hash(str(idx_oracle)))
    key_student = (hash(retrain_ds), hash(str(idx)), hash(prune_ratio))

    if key_oracle not in oracle_cache:
        oracle_tmp_val = validation.clone()
        oracle_tmp_test = test.clone()
        oracle_tmp_val.X.iloc[:, idx_oracle] = 0.
        oracle_tmp_test.X.iloc[:, idx_oracle] = 0.
        oracle_cache[key_oracle] = {
            "validation": compute_metric_percategory(oracle_tmp_val.y, oracle_net.predict(oracle_tmp_val.X), oracle_tmp_val._y, scorer=accuracy_score),
            "test": compute_metric_percategory(oracle_tmp_test.y, oracle_net.predict(oracle_tmp_test.X), oracle_tmp_test._y, scorer=accuracy_score)}

    tmp_val = validation.clone()
    tmp_test = test.clone()
    tmp_val.X.iloc[:, idx] = 0.
    tmp_test.X.iloc[:, idx] = 0.

    if key_student not in student_cache:
        student_cache[key_student] = {
            "validation": compute_metric_percategory(tmp_val.y, student_net.predict(tmp_val.X), tmp_val._y, scorer=accuracy_score),
            "test": compute_metric_percategory(tmp_test.y, student_net.predict(tmp_test.X), tmp_test._y, scorer=accuracy_score)}
    
    hs, m = student_net.fit(retrain_ds, validation_dataset=retrain_val_ds, oracle=oracle_net, idx_active_features=idx, idx_active_features_oracle=idx_oracle,
                            monitor_ds=validation, verbose=True, **kwargs)
    
    dump(m, f"{save_prefix}_monitored.csv")
    dump(hs, f"{save_prefix}_history.csv")
    
    df = pd.DataFrame(columns=cols)
    df.loc["Validation Before"] = student_cache[key_student]["validation"]
    df.loc["Validation After"] = compute_metric_percategory(tmp_val.y, student_net.predict(tmp_val.X), tmp_val._y, scorer=accuracy_score)
    df.loc["Test Before"] = student_cache[key_student]["test"]
    df.loc["Test After"] = compute_metric_percategory(tmp_test.y, student_net.predict(tmp_test.X), tmp_test._y, scorer=accuracy_score)
    df.loc["Oracle Validation"] = oracle_cache[key_oracle]["validation"]
    df.loc["Oracle Test"] = oracle_cache[key_oracle]["test"]
    dump(df, f"{save_prefix}.csv")
    return df

In [8]:
def run_scenario(scenario_name, delta_p=None, subset_ratio=None, only_client_categories=False, features_available=None, prune_ratio=None, skip=False, **kwargs):
    dirname = config.CLIENT_REFIT_OUTPUT_DIR + scenario_name + ("_few_c" if only_client_categories else "_all_c") + (f"_subsetsize_{subset_ratio}" if subset_ratio is not None else "") + (f"_prune_{prune_ratio}" if prune_ratio else "") + (f"_drop_{delta_p}" if delta_p else "") + "/"
    
    if not os.path.isdir(dirname) or skip is False:
        create_dir(dirname)
    tmp_train = train
    tmp_val = validation
    if only_client_categories:
        set_seed()
        tmp_train = tmp_train.filter_categories(config.CLIENT_CATEGORIES).balance_categories()
        tmp_val = tmp_val.filter_categories(config.CLIENT_CATEGORIES).balance_categories()
    print("----Running", len(config.SCENARIOS[scenario_name]), "combinations for", scenario_name)
    for c in config.SCENARIOS[scenario_name]:
        name = dirname + "-".join(f"{k}_{v.name if hasattr(v, 'name') else v}" for k,v in c.items())
        print(name)
        run_test(name, tmp_train, tmp_val, **kwargs, features_available=features_available, prune_ratio=prune_ratio, skip=skip, **c)

In [5]:
baselines = load(config.RANK_PRUNE_OUTPUT_DIR + f"baseline.csv", index_col=0)

In [5]:
create_dir(config.CLIENT_REFIT_OUTPUT_DIR)

## Test Refit Algoithms for different use cases

Oracle to Oracle with only client traffic categories.

In [6]:
run_scenario("o_to_o", only_client_categories=True, features_available=None, prune_ratio=None, skip=True)

----Running 4 combinations for o_to_o
refit_output/o_to_o_few_c/algorithm_ground_truth-availability_bilateral-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0022069270959026084, 'loss_validation': 0.002316353052282783, 'accuracy_score_train': 0.9953076923076923, 'accuracy_score_validation': 0.9943333333333333}: 100%|██████████| 100/100 [00:28<00:00,  3.46it/s]


refit_output/o_to_o_few_c/algorithm_ground_inferred-availability_bilateral-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0014241236642347026, 'loss_validation': 0.006573372632521398, 'accuracy_score_train': 0.9827692307692307, 'accuracy_score_validation': 0.985}: 100%|██████████| 100/100 [00:28<00:00,  3.46it/s]             


refit_output/o_to_o_few_c/algorithm_knowledge_distillation-availability_bilateral-learn_input_client-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0008253561387843412, 'loss_validation': 0.003791229344293009, 'accuracy_score_train': 0.9926923076923077, 'accuracy_score_validation': 0.9916666666666667}: 100%|██████████| 100/100 [00:29<00:00,  3.43it/s]


refit_output/o_to_o_few_c/algorithm_knowledge_distillation-availability_bilateral-learn_input_client-learn_kwargs_{'alpha': 1}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.00016858108290491812, 'loss_validation': 0.006837927505557954, 'accuracy_score_train': 0.9811538461538462, 'accuracy_score_validation': 0.9843333333333333}: 100%|██████████| 100/100 [00:29<00:00,  3.43it/s]


Oracle to Pruned Oracle. Select highest prune ratio with worst performance degradation.

In [26]:
res = load(config.RANK_PRUNE_OUTPUT_DIR + f"traffic_few_c_pruning_ratios_only_{config.PRUNE_METHOD.__name__}.csv", index_col=0)
res["Accuracy"] -= baselines.loc["Client Traffic", "Original Accuracy"]
within_range = res[res["Accuracy"].between(-0.1, 0)]
col = within_range["Prune Ratio"]
row = within_range.loc[col[col == col.max()].index[-1]]

print("Within a 10% Accuracy drop the highest prune ratio is", row["Prune Ratio"], f"(Drop of {row['Accuracy']})")
run_scenario("o_to_po", add_name="drop_10", only_client_categories=True, features_available=None, prune_ratio=row["Prune Ratio"], skip=False)

Within a 10% Accuracy drop the highest prune ratio is 0.55 (Drop of -0.05133333333333334)


----Running 4 combinations for o_to_po
refit_output/o_to_po_few_c_prune_0.55/algorithm_ground_truth-availability_bilateral-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0020448204947967753, 'loss_validation': 0.0026935815069141673, 'accuracy_score_train': 0.9957692307692307, 'accuracy_score_validation': 0.9923333333333333}: 100%|██████████| 100/100 [00:30<00:00,  3.32it/s]


refit_output/o_to_po_few_c_prune_0.55/algorithm_ground_inferred-availability_bilateral-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0013139836333583908, 'loss_validation': 0.007315250626505176, 'accuracy_score_train': 0.9822307692307692, 'accuracy_score_validation': 0.9843333333333333}: 100%|██████████| 100/100 [00:30<00:00,  3.24it/s]


refit_output/o_to_po_few_c_prune_0.55/algorithm_knowledge_distillation-availability_bilateral-learn_input_client-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0007123862970705964, 'loss_validation': 0.003990473184824633, 'accuracy_score_train': 0.9927692307692307, 'accuracy_score_validation': 0.991}: 100%|██████████| 100/100 [00:30<00:00,  3.30it/s]             


refit_output/o_to_po_few_c_prune_0.55/algorithm_knowledge_distillation-availability_bilateral-learn_input_client-learn_kwargs_{'alpha': 1}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0005622415058015678, 'loss_validation': 0.00687859117819212, 'accuracy_score_train': 0.9807692307692307, 'accuracy_score_validation': 0.983}: 100%|██████████| 100/100 [00:29<00:00,  3.34it/s]              


In [31]:
res = load(config.RANK_PRUNE_OUTPUT_DIR + f"traffic_few_c_pruning_ratios_only_{config.PRUNE_METHOD.__name__}.csv", index_col=0)
res["Accuracy"] -= baselines.loc["Client Traffic", "Original Accuracy"]
within_range = res[res["Accuracy"].between(-0.2, -0.1)]
col = within_range["Prune Ratio"]
row = within_range.loc[col[col == col.max()].index[-1]]

print("Within a 20% Accuracy drop the highest prune ratio is", row["Prune Ratio"], f"(Drop of {row['Accuracy']})")
run_scenario("o_to_po", add_name="drop_20", only_client_categories=True, features_available=None, prune_ratio=row["Prune Ratio"], skip=False)

Within a 20% Accuracy drop the highest prune ratio is 0.5 (Drop of -0.11599999999999999)


----Running 4 combinations for o_to_po
refit_output/o_to_po_few_c_prune_0.5_drop_20/algorithm_ground_truth-availability_bilateral-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0015213526654478476, 'loss_validation': 0.0028091429251471964, 'accuracy_score_train': 0.9948461538461538, 'accuracy_score_validation': 0.9926666666666667}: 100%|██████████| 100/100 [00:31<00:00,  3.18it/s]


refit_output/o_to_po_few_c_prune_0.5_drop_20/algorithm_ground_inferred-availability_bilateral-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0013094845400164065, 'loss_validation': 0.007172847272082223, 'accuracy_score_train': 0.9818461538461538, 'accuracy_score_validation': 0.984}: 100%|██████████| 100/100 [00:30<00:00,  3.25it/s]             


refit_output/o_to_po_few_c_prune_0.5_drop_20/algorithm_knowledge_distillation-availability_bilateral-learn_input_client-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0007328926222844289, 'loss_validation': 0.003955769070775684, 'accuracy_score_train': 0.9927692307692307, 'accuracy_score_validation': 0.991}: 100%|██████████| 100/100 [00:31<00:00,  3.22it/s]             


refit_output/o_to_po_few_c_prune_0.5_drop_20/algorithm_knowledge_distillation-availability_bilateral-learn_input_client-learn_kwargs_{'alpha': 1}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0004400081456609376, 'loss_validation': 0.0066844856676525535, 'accuracy_score_train': 0.9806923076923076, 'accuracy_score_validation': 0.984}: 100%|██████████| 100/100 [00:30<00:00,  3.23it/s]             


Oracle to Edge Oracle. Select the worst performant accepted feature subset.

In [19]:
for s in config.TARGET_SUBSET_RATIOS:
    res = load(config.RANK_PRUNE_OUTPUT_DIR + f"rank_few_c_traffic_few_c_subsets_features_for_subsetsize_{s}.csv", index_col=0)
    res["Accuracy"] -= baselines.loc["Client Traffic", "Original Accuracy"]
    within_range = res[res["Accuracy"].between(-0.1, 0)]
    if within_range.empty:
        print("Skipping", s, "as no data within the range")
        continue
    worst_row = within_range.iloc[-1]
    subset = worst_row[~worst_row.index.isin(["Accuracy"]) & worst_row.notnull()].index.values
    print("For a subset size of ratio", s, "the worst performant subset within 10% drop achieved an Accuracy of", worst_row["Accuracy"])
    run_scenario("o_to_eo", only_client_categories=True, features_available=subset, delta_p=0.1, subset_ratio=s, prune_ratio=None, skip=True)

Skipping 0.1 as no data within the range
For a subset size of ratio 0.3 the worst performant subset within 20% drop achieved an Accuracy of -0.09799999999999998
----Running 15 combinations for o_to_eo
refit_output/o_to_eo_few_c_subsetsize_0.3_drop_0.1/algorithm_ground_truth-availability_none-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.004451889361738448, 'loss_validation': 0.004921584376247475, 'accuracy_score_train': 0.9906153846153846, 'accuracy_score_validation': 0.989}: 100%|██████████| 100/100 [00:27<00:00,  3.59it/s]              


refit_output/o_to_eo_few_c_subsetsize_0.3_drop_0.1/algorithm_ground_inferred-availability_none-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0009716898754499468, 'loss_validation': 0.059480075604283776, 'accuracy_score_train': 0.8735384615384615, 'accuracy_score_validation': 0.8786666666666667}: 100%|██████████| 100/100 [00:27<00:00,  3.59it/s]


refit_output/o_to_eo_few_c_subsetsize_0.3_drop_0.1/algorithm_knowledge_distillation-availability_none-learn_input_client-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512


...

{'loss_train': 0.1239701031898482, 'loss_validation': 0.11997682675086531, 'accuracy_score_train': 0.7723076923076924, 'accuracy_score_validation': 0.785}: 100%|██████████| 100/100 [00:37<00:00,  2.67it/s]               


refit_output/o_to_eo_few_c_subsetsize_0.8_drop_0.1/algorithm_knowledge_distillation-availability_oracle-learn_input_mixed-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.03000973223586124, 'loss_validation': 0.03124870693711036, 'accuracy_score_train': 0.981, 'accuracy_score_validation': 0.9826666666666667}: 100%|██████████| 100/100 [00:37<00:00,  2.65it/s]              


refit_output/o_to_eo_few_c_subsetsize_0.8_drop_0.1/algorithm_knowledge_distillation-availability_oracle-learn_input_mixed-learn_kwargs_{'alpha': 1}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.11685230505897863, 'loss_validation': 0.11922690276848304, 'accuracy_score_train': 0.7356153846153846, 'accuracy_score_validation': 0.7373333333333333}: 100%|██████████| 100/100 [00:37<00:00,  2.66it/s] 


In [20]:
for s in config.TARGET_SUBSET_RATIOS:
    res = load(config.RANK_PRUNE_OUTPUT_DIR + f"rank_few_c_traffic_few_c_subsets_features_for_subsetsize_{s}.csv", index_col=0)
    res["Accuracy"] -= baselines.loc["Client Traffic", "Original Accuracy"]
    within_range = res[res["Accuracy"].between(-0.2, -0.1)]
    if within_range.empty:
        print("Skipping", s, "as no data within the range")
        continue
    worst_row = within_range.iloc[-1]
    subset = worst_row[~worst_row.index.isin(["Accuracy"]) & worst_row.notnull()].index.values
    print("For a subset size of ratio", s, "the worst performant subset within 20% drop achieved an Accuracy of", worst_row["Accuracy"])
    run_scenario("o_to_eo", only_client_categories=True, features_available=subset, delta_p=0.2, subset_ratio=s, prune_ratio=None, skip=True)

For a subset size of ratio 0.1 the worst performant subset within 20% drop achieved an Accuracy of -0.19833333333333336
----Running 15 combinations for o_to_eo
refit_output/o_to_eo_few_c_subsetsize_0.1_drop_0.2/algorithm_ground_truth-availability_none-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.005514478786100547, 'loss_validation': 0.006456528649875909, 'accuracy_score_train': 0.9869230769230769, 'accuracy_score_validation': 0.9853333333333333}: 100%|██████████| 100/100 [00:29<00:00,  3.36it/s] 


refit_output/o_to_eo_few_c_subsetsize_0.1_drop_0.2/algorithm_ground_inferred-availability_none-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.0033328680545137335, 'loss_validation': 0.10674320422022847, 'accuracy_score_train': 0.7894615384615384, 'accuracy_score_validation': 0.7753333333333333}: 100%|██████████| 100/100 [00:29<00:00,  3.35it/s]


refit_output/o_to_eo_few_c_subsetsize_0.1_drop_0.2/algorithm_knowledge_distillation-availability_none-learn_input_client-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512


...

{'loss_train': 0.1239701031898482, 'loss_validation': 0.11895551231025103, 'accuracy_score_train': 0.6536153846153846, 'accuracy_score_validation': 0.6513333333333333}: 100%|██████████| 100/100 [00:37<00:00,  2.64it/s]  


refit_output/o_to_eo_few_c_subsetsize_0.8_drop_0.2/algorithm_knowledge_distillation-availability_oracle-learn_input_mixed-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.03000973223586124, 'loss_validation': 0.04402839977544863, 'accuracy_score_train': 0.9195384615384615, 'accuracy_score_validation': 0.919}: 100%|██████████| 100/100 [00:38<00:00,  2.62it/s]              


refit_output/o_to_eo_few_c_subsetsize_0.8_drop_0.2/algorithm_knowledge_distillation-availability_oracle-learn_input_mixed-learn_kwargs_{'alpha': 1}-max_epochs_100-epochs_wo_improve_100-batch_size_512


{'loss_train': 0.11685230505897863, 'loss_validation': 0.11121578461262621, 'accuracy_score_train': 0.7176923076923077, 'accuracy_score_validation': 0.7166666666666667}: 100%|██████████| 100/100 [00:38<00:00,  2.63it/s] 


Oracle to Edge Client. Select highest prune ratio with worst performance degradation.

In [10]:
for s in config.TARGET_SUBSET_RATIOS:
    res = load(config.RANK_PRUNE_OUTPUT_DIR + f"rank_few_c_traffic_few_c_combo_pruned_models_subsets_features_for_subsetsize_{s}_{config.PRUNE_METHOD.__name__}.csv", index_col=0)
    res["Accuracy"] -= baselines.loc["Client Traffic", "Original Accuracy"]
    within_range = res[res["Accuracy"].between(-0.1, 0)]
    col = within_range["Prune Ratio"]
    worst_row = within_range.loc[col[col == col.max()].index[-1]]
    prune_ratio = worst_row["Prune Ratio"]
    subset = worst_row[~worst_row.index.isin(["Accuracy", "Prune Ratio"]) & worst_row.notnull()].index.values
    print("For a subset size of ratio", s, "the worst performant subset identified has a maximum accepted prune ratio of",
          prune_ratio, f"(Accuracy of {worst_row['Accuracy']})")
    run_scenario("o_to_ec", only_client_categories=True, features_available=subset, delta_p=0.1, subset_ratio=s, prune_ratio=prune_ratio, skip=True)

For a subset size of ratio 0.1 the worst performant subset identified has a maximum accepted prune ratio of 0.25 (Accuracy of -0.09033333333333327)
----Running 15 combinations for o_to_ec
refit_output/o_to_ec_few_c_subsetsize_0.1_prune_0.25_drop_0.1/algorithm_ground_truth-availability_none-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512
refit_output/o_to_ec_few_c_subsetsize_0.1_prune_0.25_drop_0.1/algorithm_ground_inferred-availability_none-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512
refit_output/o_to_ec_few_c_subsetsize_0.1_prune_0.25_drop_0.1/algorithm_knowledge_distillation-availability_none-learn_input_client-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512
refit_output/o_to_ec_few_c_subsetsize_0.1_prune_0.25_drop_0.1/algorithm_knowledge_distillation-availability_none-learn_input_client-learn_kwargs_{'alpha': 1}-max_epochs_100-epochs_wo_improve_100-batch_size_512
refit_output/o_to_ec_few_c_subsetsize_0

In [11]:
for s in config.TARGET_SUBSET_RATIOS:
    res = load(config.RANK_PRUNE_OUTPUT_DIR + f"rank_few_c_traffic_few_c_combo_pruned_models_subsets_features_for_subsetsize_{s}_{config.PRUNE_METHOD.__name__}.csv", index_col=0)
    res["Accuracy"] -= baselines.loc["Client Traffic", "Original Accuracy"]
    within_range = res[res["Accuracy"].between(-0.2, -0.1)]
    col = within_range["Prune Ratio"]
    worst_row = within_range.loc[col[col == col.max()].index[-1]]
    prune_ratio = worst_row["Prune Ratio"]
    subset = worst_row[~worst_row.index.isin(["Accuracy", "Prune Ratio"]) & worst_row.notnull()].index.values
    print("For a subset size of ratio", s, "the worst performant subset identified has a maximum accepted prune ratio of",
          prune_ratio, f"(Accuracy of {worst_row['Accuracy']})")
    run_scenario("o_to_ec", only_client_categories=True, features_available=subset, delta_p=0.2, subset_ratio=s, prune_ratio=prune_ratio, skip=True)

For a subset size of ratio 0.1 the worst performant subset identified has a maximum accepted prune ratio of 0.6 (Accuracy of -0.19033333333333335)
----Running 15 combinations for o_to_ec
refit_output/o_to_ec_few_c_subsetsize_0.1_prune_0.6_drop_0.2/algorithm_ground_truth-availability_none-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512
refit_output/o_to_ec_few_c_subsetsize_0.1_prune_0.6_drop_0.2/algorithm_ground_inferred-availability_none-learn_input_client-max_epochs_100-epochs_wo_improve_100-batch_size_512
refit_output/o_to_ec_few_c_subsetsize_0.1_prune_0.6_drop_0.2/algorithm_knowledge_distillation-availability_none-learn_input_client-learn_kwargs_{'alpha': 0.5}-max_epochs_100-epochs_wo_improve_100-batch_size_512
refit_output/o_to_ec_few_c_subsetsize_0.1_prune_0.6_drop_0.2/algorithm_knowledge_distillation-availability_none-learn_input_client-learn_kwargs_{'alpha': 1}-max_epochs_100-epochs_wo_improve_100-batch_size_512
refit_output/o_to_ec_few_c_subsetsize_0.1_pr

If interested in any further test, there are all those with the model whose feature ranking has been performed on all traffic categories, and also those where the stochastic search is performed before the pruning (expect the oracle to be better than pruned model always).